### Урок 7. Сверточные нейронные сети для анализа текста

Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1.  Учим conv сеть для классификации
2.  Рассмотреть 2-а варианта сеточек<br>
    2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>
    2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [2]:
# Затянуть датафреймы.

import os
import pandas as pd

In [3]:
# !pip install xlrd

In [4]:
df = pd.read_excel('отзывы за лето.xls')

In [5]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
df = df[['Rating','Content']]
df

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."
...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра..."
20655,5,Ок
20656,4,Доволен
20657,1,"Песопаснасть, рут ни нужын"


In [7]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Установи порог отзывов 3 бала. если значение выше порога отзыв положительный. Если ниже- отрицательный.

In [8]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [9]:
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

### Выполнить предобработку текста.

In [10]:
max_words = 1000
max_len = 100
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

Обучить сверточную нейронную сеть.

Как в прошлом занятии.

In [11]:
# !pip install stop_words

In [12]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py


In [13]:
# Теперь повторим это для всех записей.
def replase_words(text,dict_): 
    output = ''
    for word in text.split(' '): # не будем делить текст на части будем искать подстроку в строке. это касается только смайликов.
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

In [14]:
df['Content_с'] = df['Content']

In [15]:
df

,Rating,Content,Content_с
0,1,It just works!,It just works!
1,1,В целом удобноное приложение...из минусов хотя...,В целом удобноное приложение...из минусов хотя...
2,1,Отлично все,Отлично все
3,1,Стал зависать на 1% работы антивируса. Дальше ...,Стал зависать на 1% работы антивируса. Дальше ...
4,1,"Очень удобно, работает быстро.","Очень удобно, работает быстро."
...,...,...,...
20654,0,"Ну и шляпа,с роот правами бесполезная прога,ра...","Ну и шляпа,с роот правами бесполезная прога,ра..."
20655,1,Ок,Ок
20656,1,Доволен,Доволен
20657,0,"Песопаснасть, рут ни нужын","Песопаснасть, рут ни нужын"


In [16]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    # Заменим эмодзи на соответствующие им слова.
    txt = re.sub("[\,]","",txt)
    txt = replase_words(txt, emoticon_dict)
     # Заменим сокращения на их полные формы
    txt = replase_words(txt, apostrophe_dict)
    txt = replase_words(txt, short_word_dict)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split()] # if word not in sw]
    # [w for w in text.split() if len(w)>1 and w != "i"])
    return " ".join([w for w in txt if len(w)>1])

df['Content'] = df['Content'].apply(preprocess_text)

После очитски текста, длина части комментариев оказалась равной нулю!

In [17]:
df['Text_len'] = [len(text) for text in df['Content']]

In [18]:
df.loc[((df['Text_len']==0) & (df['Rating']==0))]

,Rating,Content,Content_с,Text_len


In [19]:
df.loc[((df['Text_len']==0) & (df['Rating']==1))]

,Rating,Content,Content_с,Text_len
3080,1,,.👍,0
4083,1,,+,0
7479,1,,👍!!!,0
8724,1,,О.,0
15887,1,,N 1,0
16207,1,,во,0


In [20]:
df.loc[(df['Text_len']==0),'Rating'].value_counts() # как быть вот с такими комметариями!

1    6
Name: Rating, dtype: int64

Можно посмотреть что это были за комментарии и выполнить очитску более аккуратно, чтобы не стереть ничего лишнего.

Раделим df на ренировочную и тестовую выборки !!! 

In [21]:
df_train, df_test = train_test_split(df,test_size=0.3)

In [22]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [23]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
tokens_filtered = [word for word in tokens if word.isalnum()]

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [25]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [26]:
tokens_filtered_top[:50]

['приложение',
 'всё',
 'очень',
 'удобно',
 'на',
 'работать',
 'удобный',
 'что',
 'отлично',
 'спасибо',
 'хороший',
 'нравиться',
 'это',
 'отличный',
 'раз',
 'по',
 'хорошо',
 'после',
 'но',
 'телефон',
 'супер',
 'быть',
 'мочь',
 'за',
 'при',
 'обновление',
 'быстро',
 'пароль',
 'просто',
 'так',
 'только',
 'не',
 'как',
 'пользоваться',
 'банк',
 'антивирус',
 'сбербанк',
 'вход',
 'устраивать',
 'вы',
 'для',
 'карта',
 'то',
 'пока',
 'прошивка',
 'свой',
 'из',
 'весь',
 'без',
 'рута']

In [27]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [28]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [29]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
# x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

# Keras model

In [30]:
# !pip install keras

In [31]:
import numpy as np
import tensorflow.keras

In [32]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 

from tensorflow.keras.callbacks import EarlyStopping  
from sklearn.metrics import accuracy_score 

In [70]:
# from tensorflow.keras.objectives import categorical_crossentropy

In [112]:
num_classes = 2
y_train = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)

In [77]:
# GlobalMaxPool1D?

In [113]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [116]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [117]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 13014 samples, validate on 1447 samples
Epoch 1/20
13014/13014 [==============================] - 10s 751us/sample - loss: 0.4806 - acc: 0.8064 - val_loss: 0.3886 - val_acc: 0.8037
Epoch 2/20
13014/13014 [==============================] - 8s 649us/sample - loss: 0.3216 - acc: 0.8064 - val_loss: 0.2806 - val_acc: 0.8037
Epoch 3/20
13014/13014 [==============================] - 6s 481us/sample - loss: 0.2488 - acc: 0.8251 - val_loss: 0.2531 - val_acc: 0.8970
Epoch 4/20
13014/13014 [==============================] - 8s 597us/sample - loss: 0.2263 - acc: 0.9167 - val_loss: 0.2479 - val_acc: 0.8998
Epoch 5/20
13014/13014 [==============================] - 9s 685us/sample - loss: 0.2148 - acc: 0.9277 - val_loss: 0.2495 - val_acc: 0.8998


In [118]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

6198/6198 [==============================] - 1s 83us/sample


In [119]:
y_pred = np.argmax(results,axis = 1)

In [122]:
accuracy_score(df_test['Rating'].to_numpy(), y_pred)

0.9101323007421749

## Задание 2.

2.  Рассмотреть 2-а варианта сеточек<br>
    2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>
    2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

пример классификации текста с помощю библиотеки взят отсюда -- https://github.com/ltgoslo/simple_elmo/blob/master/simple_elmo/examples/text_classification.py